READING DATA FROM API AND WRITING TO DB

In [38]:
import openmeteo_requests
import requests
import pandas as pd
from pymongo import MongoClient
from bson import json_util
import json
import requests_cache
import pandas as pd
from retry_requests import retry

client = MongoClient("mongodb://%s:%s@127.0.0.1" % ("dap", "dap"))
db = client['TRY']
collection = db['weather1']
# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
	"latitude": 53.3331,
	"longitude": -6.2489,
	"start_date": "2023-01-01",
	"end_date": "2023-12-31",
	"hourly": ["temperature_2m", "relative_humidity_2m", "rain", "snowfall", "weather_code", "wind_speed_10m", "is_day", "sunshine_duration"],
	"timezone": "GMT"
}
responses = openmeteo.weather_api(url, params=params)
response = responses[0]
# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
hourly_relative_humidity_2m = hourly.Variables(1).ValuesAsNumpy()
hourly_rain = hourly.Variables(2).ValuesAsNumpy()
hourly_snowfall = hourly.Variables(3).ValuesAsNumpy()
hourly_weather_code = hourly.Variables(4).ValuesAsNumpy()
hourly_wind_speed_10m = hourly.Variables(5).ValuesAsNumpy()
hourly_is_day = hourly.Variables(6).ValuesAsNumpy()
hourly_sunshine_duration = hourly.Variables(7).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}
hourly_data["temperature_2m"] = hourly_temperature_2m
hourly_data["relative_humidity_2m"] = hourly_relative_humidity_2m
hourly_data["rain"] = hourly_rain
hourly_data["snowfall"] = hourly_snowfall
hourly_data["weather_code"] = hourly_weather_code
hourly_data["wind_speed_10m"] = hourly_wind_speed_10m
hourly_data["is_day"] = hourly_is_day
hourly_data["sunshine_duration"] = hourly_sunshine_duration

hourly_dataframe = pd.DataFrame(data = hourly_data)
data_dict = hourly_dataframe.to_dict('records')
collection.insert_many(data_dict)


InsertManyResult([ObjectId('66205b475e9a1fd2eefb1f62'), ObjectId('66205b475e9a1fd2eefb1f63'), ObjectId('66205b475e9a1fd2eefb1f64'), ObjectId('66205b475e9a1fd2eefb1f65'), ObjectId('66205b475e9a1fd2eefb1f66'), ObjectId('66205b475e9a1fd2eefb1f67'), ObjectId('66205b475e9a1fd2eefb1f68'), ObjectId('66205b475e9a1fd2eefb1f69'), ObjectId('66205b475e9a1fd2eefb1f6a'), ObjectId('66205b475e9a1fd2eefb1f6b'), ObjectId('66205b475e9a1fd2eefb1f6c'), ObjectId('66205b475e9a1fd2eefb1f6d'), ObjectId('66205b475e9a1fd2eefb1f6e'), ObjectId('66205b475e9a1fd2eefb1f6f'), ObjectId('66205b475e9a1fd2eefb1f70'), ObjectId('66205b475e9a1fd2eefb1f71'), ObjectId('66205b475e9a1fd2eefb1f72'), ObjectId('66205b475e9a1fd2eefb1f73'), ObjectId('66205b475e9a1fd2eefb1f74'), ObjectId('66205b475e9a1fd2eefb1f75'), ObjectId('66205b475e9a1fd2eefb1f76'), ObjectId('66205b475e9a1fd2eefb1f77'), ObjectId('66205b475e9a1fd2eefb1f78'), ObjectId('66205b475e9a1fd2eefb1f79'), ObjectId('66205b475e9a1fd2eefb1f7a'), ObjectId('66205b475e9a1fd2eefb1f

In [39]:
hourly_dataframe

,date,temperature_2m,relative_humidity_2m,rain,snowfall,weather_code,wind_speed_10m,is_day,sunshine_duration
0,2023-01-01 00:00:00+00:00,6.27,91.387459,0.0,0.0,3.0,17.935081,0.0,0.0
1,2023-01-01 01:00:00+00:00,5.57,90.385803,0.1,0.0,51.0,18.504139,0.0,0.0
2,2023-01-01 02:00:00+00:00,4.72,92.905891,0.0,0.0,2.0,17.566378,0.0,0.0
3,2023-01-01 03:00:00+00:00,4.92,93.571838,0.0,0.0,3.0,16.808571,0.0,0.0
4,2023-01-01 04:00:00+00:00,5.47,91.976448,0.0,0.0,2.0,16.299694,0.0,0.0
...,...,...,...,...,...,...,...,...,...
8755,2023-12-31 19:00:00+00:00,7.12,85.905251,0.1,0.0,51.0,28.162956,0.0,0.0
8756,2023-12-31 20:00:00+00:00,6.97,86.189529,0.2,0.0,51.0,27.153164,0.0,0.0
8757,2023-12-31 21:00:00+00:00,6.57,87.056412,0.0,0.0,2.0,27.047956,0.0,0.0
8758,2023-12-31 22:00:00+00:00,6.42,86.434845,0.0,0.0,3.0,27.698952,0.0,0.0


In [ ]:
# Process first location. Add a for-loop for multiple locations or weather models

print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")


print(hourly_dataframe)


In [ ]:
import pandas as pd
df = pd.read_csv('D://DAP//TeamAA_DAP_JAN2024//pedcount.csv')
collection_ped = db['ped_count']
data = df.to_dict('records')
collection_ped.insert_many(data)